In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix

import numpy as np

path = "../Data/"
words = []


def train_to_sentence_form(path):
    # reads the file to a format of docs[doc[[sentence1],[sentence2]],...]
    docs=[]
    num_of_sentences = []
    with open(path) as f:
        line = f.readline()
        while line:
            line = line.rstrip('\n').split(" ")
            #print(line)
            num = line.pop(0)
            #print(num)
            number = int(num[1:-1])
            num_of_sentences.append(number)
            len_of_sentence = []

            for i,k in enumerate(line):
                if k.startswith('<'):
                    k=k[1:-1]
                    len_of_sentence.append(int(k))
                    line.pop(i)
            doc_sentence=[]       
            sentence=[]
            for j in len_of_sentence:
                sentence = line[:j]
                line=line[j:]
                docs.append( sentence) 
            line = f.readline()
    print(len(docs))
    return docs,num_of_sentences     
    #docs is a list of lists where each list is a different sentence





vocabs = open(path+"vocabs.txt").readlines()
for vocab in vocabs:
    word, _ = vocab.split(', ')
    words.append(word)

train_label = [i.strip() for i in open(path+"train-label.dat").readlines()]
train_label = np.array(train_label)

#path="../Data/"
docs,num_sentences = train_to_sentence_form(path+"train-data.dat")

#train_data = convert_to_word(path + "/train-data.dat")
data = []
for i in docs:
    final_L = ''
    for l in i:
        final_L = final_L + ' ' + words[int(l)]
    data.append(final_L[1:])
print(data[:12])
print(len(data))       
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(data)
print(X_train_counts.size)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)




149925
['rubi rail helper demo more info auto complet', 'see new helper action', 'null length substr locat', 'exec messag messag pleas edit remov follow word content', 'roll stone com news song previou next page', 'good vibrat beach boy', 'smell teen spirit nirvana', 'want hold hand beatl', 'hound dog elvi', 'god know beach boy', 'walk line johnni cash', 'heaven led zeppelin']
149925
965852


In [2]:
def multilabel_confusion_matrix(predict,path,clustnum=-3):
    test_label = [i.strip().split() for i in open(path + "test-label.dat").readlines()]

    for i in range(len(test_label)):
        item = test_label[i]
        for j in range(len(item)):
            item[j] = int(item[j])

        test_label[i] = item

    labels = []

    for item in open(path + "labels.txt").readlines():
        item = item.replace('\n', '')
        label_name, _ = item.split(', ')
        labels.append(label_name)

    y_true = np.array(test_label)
    y_pred = np.array(predict)

    conf_mat_dict = {}

    for label_col in range(len(labels)):
        y_true_label = y_true[:, label_col]
        y_pred_label = y_pred[:, label_col]
        conf_mat_dict[labels[label_col]] = confusion_matrix(y_pred=y_pred_label, y_true=y_true_label)
    f = open("confmatrix"+str(clustnum)+".txt", "w")
    f.close()
    for label, matrix in conf_mat_dict.items():
        print("Confusion matrix for label {}:".format(label))
        f = open("confmatrix"+str(clustnum)+".txt", "a")
        print(matrix)
        f.write("Confusion matrix for label {}:".format(label))
        np.savetxt(f,matrix)
        f.close()
    return

In [3]:
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
# KMeans clustering a kind of clustering.
from sklearn.cluster import KMeans


test_data,test_sentences = train_to_sentence_form(path+"test-data.dat")
test_label = [i.strip() for i in open(path+"test-label.dat").readlines()]
test_label = np.array(test_label)
train_label = [i.strip() for i in open(path+"train-label.dat").readlines()]
train_label = np.array(train_label)

df = pd.read_csv("../multi-instance/k_means/train_kmeans_svd.csv") 
df_test = pd.read_csv("../multi-instance/k_means/test_kmeans_svd.csv")
#train_label = pd.DataFrame(data = train_label)
labels_models=[]
for i in range (0,20 ):
    tr=[]
    for j in range (0,len(train_label)):
        tr.append(int(train_label[j][2*i]))
    labels_models.append(tr)
    
train_labels = pd.DataFrame(labels_models)
train_labels = train_labels.transpose()
#print(train_labels[:][2])
labels__models=[]
for i in range (0,20 ):
    ts=[]
    for j in range (0,len(test_label)):
        ts.append(int(test_label[j][2*i]))
    labels__models.append(ts)
    
test_labels = pd.DataFrame(labels__models)
test_labels = test_labels.transpose()

#print(df)
#print(df_test)
#df=df.drop(df.columns[0],axis=1)
#df_test=df_test.drop(df_test.columns[0],axis=1)

from sklearn import preprocessing
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
x = df_test.values #returns a numpy array
x_scaled = min_max_scaler.transform(x)
df_test = pd.DataFrame(x_scaled)

print(df)
print(df_test)

from sklearn import neighbors
pred = pd.DataFrame()
sm = SMOTE(sampling_strategy=0.5, random_state=2)
smsmall = SMOTE(sampling_strategy=0.25, random_state=2)
rm = RandomUnderSampler(random_state=42)
dfinit = df.copy()
orderlist = [2,7,4,0,8,6,5,12,10,9,17,11,15,14,3,13,16,1,18,19]
for l in orderlist:
    print(train_labels[train_labels[:][l] == 1][l].size)
    if train_labels[train_labels[:][l] == 1][l].size < 1000:
        print("kkk")
        df, train_labels_new = smsmall.fit_resample(dfinit,train_labels[:][l])
        df, train_labels_new = rm.fit_resample(df,train_labels_new)
    elif train_labels[train_labels[:][l] == 1][l].size < 2000:
        df, train_labels_new = sm.fit_resample(dfinit,train_labels[:][l])
        df, train_labels_new = rm.fit_resample(df,train_labels_new)
    else:
        df, train_labels_new = rm.fit_resample(dfinit, train_labels[:][l])
    print(df.shape)
    KNN = neighbors.KNeighborsClassifier(n_neighbors=3 ,weights='distance',metric='minkowski',p=1).fit(df, train_labels_new)
    dfinit['Label'+str(l)] = train_labels[:][l]
    df_test['Label'+str(l)] = KNN.predict(df_test) 
    pred['Label'+str(l)] = df_test['Label'+str(l)]
    

#clustnum = -1 smote
#-2 randomundersampler
'''sm = SMOTE(sampling_strategy='not majority', random_state=2)
df, train_labels = sm.fit_resample(df, train_labels)


KNN = neighbors.KNeighborsClassifier(n_neighbors=3 ,weights='distance',metric='minkowski',p=1).fit(df, train_labels)'''


#pred = KNN.predict(df_test)
#print(pred)
predict=[]
for p in range(3983):
    #print(p)
    result = []
    for r in range(20):
        x = orderlist.index(r)
        result.append(pred.iloc[p,x])
    predict.append(result)
    
multilabel_confusion_matrix(predict,path)

73363
             0     1         2    3      4         5         6         7  \
0     0.000000  0.00  0.000000  0.0  0.000  0.000000  0.076923  0.000000   
1     0.038462  0.00  0.133333  0.0  0.000  0.741935  0.076923  0.000000   
2     0.000000  0.12  0.333333  0.0  0.000  0.387097  0.000000  0.000000   
3     0.000000  0.00  0.000000  0.0  0.000  0.064516  0.000000  0.000000   
4     0.000000  0.08  0.000000  0.0  0.125  0.032258  0.000000  0.000000   
...        ...   ...       ...  ...    ...       ...       ...       ...   
8246  0.000000  0.00  0.000000  0.0  0.000  0.225806  0.000000  0.181818   
8247  0.000000  0.00  0.266667  0.0  0.000  0.645161  0.000000  0.000000   
8248  0.000000  0.00  0.000000  0.0  0.000  0.677419  0.230769  0.272727   
8249  0.000000  0.00  0.000000  0.0  0.000  0.096774  0.000000  0.000000   
8250  0.000000  0.00  0.000000  0.0  0.000  0.032258  0.076923  0.000000   

         8    9        10        11   12   13   14        15   16   17  \
0     0